<a href="https://colab.research.google.com/github/bowcai/YOLOv3/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run the following block if in Google Colab.

In [1]:
import os
from subprocess import getoutput
clone_output = getoutput("git clone https://github.com/bowcai/YOLOv3.git")
print(clone_output)
os.chdir('YOLOv3')

Cloning into 'YOLOv3'...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from yolov3.load_data import create_training_instances, BatchGenerator
from yolov3.load_weight import load_keras_model, save_keras_model, load_pretrained_weight
from yolov3.model import create_yolov3_model
from yolov3.train import fit_model
from yolov3.evaluate import evaluate

In [4]:
# The path of training and validation set.
proj_folder = '/content/drive/My Drive/Colab Notebooks/YOLOv3-proj/'
train_annot_folder = proj_folder + 'kangaroo/annots/'
train_img_folder = proj_folder + 'kangaroo/images/'
train_cache_dir = proj_folder + 'kangaroo/cache.pkl'
valid_annot_folder = None
valid_img_folder = None
valid_cache_dir = None

# The labels of object classes.
labels = ['kangaroo']

# Batch size.
batch_size = 16

# Warmup batches.
warmup_batches = 3

# If the IoU of predicted bounding box with any ground truth box is higher than the threshold,
# ignore the loss of the predicted box.
ignore_thresh = 0.5

grid_scales = [1, 1, 1]
obj_scale = 5
noobj_scale = 1
xywh_scale = 1
class_scale = 1

# Minimum and maximum input size.
min_input_size = 288
max_input_size = 448

# The pre-defined anchor sizes.
anchors = [55, 69, 75, 234, 133, 240, 136, 129, 142, 363, 203, 290, 228, 184, 285, 359, 341, 260]

# Ratio between network input's size and network output's size, 32 for YOLOv3.
downsample = 32

# The path of pre-trained YOLOv3 weights.
pretrained_weight_path = proj_folder + 'weights/yolov3.weights'

# The path of trained Keras model.
keras_model_path = proj_folder + 'weights/trained_model.h5'


Create training and validation set.

In [5]:
train_inst, valid_inst, labels, max_box_per_image = create_training_instances(
    train_annot_folder,
    train_img_folder,
    train_cache_dir,
    valid_annot_folder,
    valid_img_folder,
    valid_cache_dir,
    labels
)

valid_annot_folder not exists. Spliting the trainining set.
Seen labels: 	{'kangaroo': 266}

Given labels: 	['kangaroo']


Create the mini-batch generators.

In [6]:
train_generator = BatchGenerator(
    instances=train_inst,
    anchors=anchors,
    labels=labels,
    downsample=downsample,
    max_box_per_image=max_box_per_image,
    batch_size=batch_size,
    min_net_size=min_input_size,
    max_net_size=max_input_size,
    shuffle=True,
    jitter=0.3,
    norm=True
)

valid_generator = BatchGenerator(
    instances=valid_inst,
    anchors=anchors,
    labels=labels,
    downsample=downsample,
    max_box_per_image=max_box_per_image,
    batch_size=batch_size,
    min_net_size=min_input_size,
    max_net_size=max_input_size,
    shuffle=True,
    jitter=0.0,
    norm=True
)


In [7]:
# If trained model exists, load the model. Otherwise, create a new model.
if os.path.isfile(keras_model_path):
    train_model = load_keras_model(keras_model_path)

else:
    train_model, infer_model = create_yolov3_model(
        nb_class=len(labels),
        anchors=anchors,
        max_box_per_image=max_box_per_image,
        max_grid=[max_input_size, max_input_size],
        batch_size=batch_size,
        warmup_batches=warmup_batches,
        ignore_thresh=ignore_thresh,
        grid_scales=grid_scales,
        obj_scale=obj_scale,
        noobj_scale=noobj_scale,
        xywh_scale=xywh_scale,
        class_scale=class_scale,
    )

    # Load the pre-trained weights into the training model.
    load_pretrained_weight(train_model, pretrained_weight_path, keras_model_path)

loading weights of convolution #0
loading weights of convolution #1
loading weights of convolution #2
loading weights of convolution #3
no convolution #4
loading weights of convolution #5
loading weights of convolution #6
loading weights of convolution #7
no convolution #8
loading weights of convolution #9
loading weights of convolution #10
no convolution #11
loading weights of convolution #12
loading weights of convolution #13
loading weights of convolution #14
no convolution #15
loading weights of convolution #16
loading weights of convolution #17
no convolution #18
loading weights of convolution #19
loading weights of convolution #20
no convolution #21
loading weights of convolution #22
loading weights of convolution #23
no convolution #24
loading weights of convolution #25
loading weights of convolution #26
no convolution #27
loading weights of convolution #28
loading weights of convolution #29
no convolution #30
loading weights of convolution #31
loading weights of convolution #32

loading weights of convolution #81
no convolution #82
no convolution #83
loading weights of convolution #84
no convolution #85
no convolution #86
loading weights of convolution #87
loading weights of convolution #88
loading weights of convolution #89
loading weights of convolution #90
loading weights of convolution #91
loading weights of convolution #92
loading weights of convolution #93
no convolution #94
no convolution #95
loading weights of convolution #96
no convolution #97
no convolution #98
loading weights of convolution #99
loading weights of convolution #100
loading weights of convolution #101
loading weights of convolution #102
loading weights of convolution #103
loading weights of convolution #104
loading weights of convolution #105


In [8]:
# Fit the model.
fit_model(train_model, train_generator, valid_generator)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


resizing:  352 352
Epoch 1/10


resizing:  320 320
9/9 - 50s - loss: 617.8804 - yolo_layer_loss: 396.9948 - yolo_layer_1_loss: 73.0721 - yolo_layer_2_loss: 147.8135 - 50s/epoch - 6s/step
Epoch 2/10
resizing:  416 416
resizing:  352 352
9/9 - 14s - loss: 352.0565 - yolo_layer_loss: 119.1094 - yolo_layer_1_loss: 76.6596 - yolo_layer_2_loss: 156.2875 - 14s/epoch - 2s/step
Epoch 3/10
resizing:  448 448
resizing:  320 320
9/9 - 17s - loss: 334.8260 - yolo_layer_loss: 86.2128 - yolo_layer_1_loss: 80.0934 - yolo_layer_2_loss: 168.5199 - 17s/epoch - 2s/step
Epoch 4/10
resizing:  352 352
resizing:  320 320
9/9 - 6s - loss: 240.1746 - yolo_layer_loss: 43.6890 - yolo_layer_1_loss: 61.1953 - yolo_layer_2_loss: 135.2902 - 6s/epoch - 657ms/step
Epoch 5/10
resizing:  352 352
resizing:  288 288
9/9 - 5s - loss: 210.4313 - yolo_layer_loss: 38.5866 - yolo_layer_1_loss: 50.9351 - yolo_layer_2_loss: 120.9095 - 5s/epoch - 600ms/step
Epoch 6/10
resizing:  448 448
9/9 - 11s - loss: 174.3212 - yolo_layer_loss: 31.9854 - yolo_layer_1_loss: 3

In [9]:
save_keras_model(train_model, keras_model_path)

In [10]:
infer_model = load_keras_model(keras_model_path)

ValueError: ignored

In [ ]:
average_precisions = evaluate(infer_model, valid_generator)

In [ ]:
for label, average_precision in average_precisions.items():
    print(labels[label] + ': {:.4f}'.format(average_precision))
print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))